In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt


In [3]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성
    model_name="gpt-4.1-nano",  # 모델명
)

In [4]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

examples = [
    {
        "question": "BTS의 리더가 태어난 도시의 시장은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: BTS의 리더는 누구인가요?
  중간 답변: BTS의 리더는 RM(김남준)입니다.
  추가 질문: RM은 어느 도시에서 태어났나요?
  중간 답변: RM은 일산에서 태어났습니다.
  추가 질문: 일산이 속한 도시는 어디인가요?
  중간 답변: 일산은 고양시에 속합니다.
  추가 질문: 현재 고양시장은 누구인가요?
  중간 답변: 현재 고양시장은 이동환입니다.
  최종 답변은: 이동환
  """,
    },
    {
        "question": "아이유와 수지 중 누가 먼저 데뷔했나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 아이유는 언제 데뷔했나요?
  중간 답변: 아이유는 2008년에 데뷔했습니다.
  추가 질문: 수지는 언제 데뷔했나요?
  중간 답변: 수지는 미쓰에이로 2010년에 데뷔했습니다.
  최종 답변은: 아이유
  """,
    },
    {
        "question": "삼성전자 창업자의 아들이 졸업한 대학의 소재지는 어디인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 삼성전자의 창업자는 누구인가요?
  중간 답변: 삼성전자의 창업자는 이병철입니다.
  추가 질문: 이병철의 아들은 누구인가요?
  중간 답변: 이병철의 아들 중 대표적인 인물은 이건희입니다.
  추가 질문: 이건희는 어느 대학을 졸업했나요?
  중간 답변: 이건희는 와세다대학을 졸업했습니다.
  추가 질문: 와세다대학은 어디에 있나요?
  중간 답변: 와세다대학은 일본 도쿄에 있습니다.
  최종 답변은: 일본 도쿄
  """,
    },
    {
        "question": "2002년 월드컵 개최국들의 현재 수도는 같은가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 2002년 월드컵은 어느 나라에서 개최되었나요?
  중간 답변: 2002년 월드컵은 한국과 일본이 공동 개최했습니다.
  추가 질문: 한국의 수도는 어디인가요?
  중간 답변: 한국의 수도는 서울입니다.
  추가 질문: 일본의 수도는 어디인가요?
  중간 답변: 일본의 수도는 도쿄입니다.
  추가 질문: 서울과 도쿄는 같은 도시인가요?
  중간 답변: 아니요, 서울과 도쿄는 다른 도시입니다.
  최종 답변은: 아니요
  """,
    },
    {
        "question": "태극기를 디자인한 사람이 살았던 시대의 왕조는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 태극기를 디자인한 사람은 누구인가요?
  중간 답변: 태극기는 박영효가 디자인했습니다.
  추가 질문: 박영효는 언제 살았나요?
  중간 답변: 박영효는 1861년부터 1939년까지 살았습니다.
  추가 질문: 1861년부터 1939년 사이 한국의 왕조는 무엇이었나요?
  중간 답변: 1861년부터 1910년까지는 조선왕조, 1910년부터 1939년까지는 일제강점기였습니다.
  최종 답변은: 조선왕조 (주로 활동한 시기 기준)
  """,
    },
]

In [5]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))

Question:
BTS의 리더가 태어난 도시의 시장은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: BTS의 리더는 누구인가요?
  중간 답변: BTS의 리더는 RM(김남준)입니다.
  추가 질문: RM은 어느 도시에서 태어났나요?
  중간 답변: RM은 일산에서 태어났습니다.
  추가 질문: 일산이 속한 도시는 어디인가요?
  중간 답변: 일산은 고양시에 속합니다.
  추가 질문: 현재 고양시장은 누구인가요?
  중간 답변: 현재 고양시장은 이동환입니다.
  최종 답변은: 이동환
  


In [6]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "하정우 AI수석과 배우 하정우는 누가 더 나이가 많나요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question:
BTS의 리더가 태어난 도시의 시장은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: BTS의 리더는 누구인가요?
  중간 답변: BTS의 리더는 RM(김남준)입니다.
  추가 질문: RM은 어느 도시에서 태어났나요?
  중간 답변: RM은 일산에서 태어났습니다.
  추가 질문: 일산이 속한 도시는 어디인가요?
  중간 답변: 일산은 고양시에 속합니다.
  추가 질문: 현재 고양시장은 누구인가요?
  중간 답변: 현재 고양시장은 이동환입니다.
  최종 답변은: 이동환
  

Question:
아이유와 수지 중 누가 먼저 데뷔했나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 아이유는 언제 데뷔했나요?
  중간 답변: 아이유는 2008년에 데뷔했습니다.
  추가 질문: 수지는 언제 데뷔했나요?
  중간 답변: 수지는 미쓰에이로 2010년에 데뷔했습니다.
  최종 답변은: 아이유
  

Question:
삼성전자 창업자의 아들이 졸업한 대학의 소재지는 어디인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 삼성전자의 창업자는 누구인가요?
  중간 답변: 삼성전자의 창업자는 이병철입니다.
  추가 질문: 이병철의 아들은 누구인가요?
  중간 답변: 이병철의 아들 중 대표적인 인물은 이건희입니다.
  추가 질문: 이건희는 어느 대학을 졸업했나요?
  중간 답변: 이건희는 와세다대학을 졸업했습니다.
  추가 질문: 와세다대학은 어디에 있나요?
  중간 답변: 와세다대학은 일본 도쿄에 있습니다.
  최종 답변은: 일본 도쿄
  

Question:
2002년 월드컵 개최국들의 현재 수도는 같은가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 2002년 월드컵은 어느 나라에서 개최되었나요?
  중간 답변: 2002년 월드컵은 한국과 일본이 공동 개최했습니다.
  추가 질문: 한국의 수도는 어디인가요?
  중간 답변: 한국의 수도는 서울

In [14]:
# 결과 출력
answer = llm.stream(final_prompt)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: 하정우 AI수석은 누구인가요?  
중간 답변: 하정우 AI수석은 인공지능 관련 직책이나 역할을 맡은 인물입니다.  
추가 질문: 배우 하정우는 누구인가요?  
중간 답변: 배우 하정우는 대한민국의 배우입니다.  
추가 질문: 두 인물의 출생년도를 알려줄 수 있나요?  
중간 답변: 배우 하정우는 1978년에 태어났습니다.  
추가 질문: 하정우 AI수석의 출생년도를 알 수 있나요?  
중간 답변: 해당 정보가 공개되어 있지 않거나 명확하지 않습니다.  
최종 답변은: 배우 하정우가 더 나이가 많습니다.

In [7]:
# question = "하정우 AI수석과 배우 하정우는 누가 더 나이가 많나요?"
question = "네이버에서 인공지능 연구를 이끌었던 하정수 수석과 배우 하정우는 누가 더 나이가 많나요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question:
BTS의 리더가 태어난 도시의 시장은 누구인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: BTS의 리더는 누구인가요?
  중간 답변: BTS의 리더는 RM(김남준)입니다.
  추가 질문: RM은 어느 도시에서 태어났나요?
  중간 답변: RM은 일산에서 태어났습니다.
  추가 질문: 일산이 속한 도시는 어디인가요?
  중간 답변: 일산은 고양시에 속합니다.
  추가 질문: 현재 고양시장은 누구인가요?
  중간 답변: 현재 고양시장은 이동환입니다.
  최종 답변은: 이동환
  

Question:
아이유와 수지 중 누가 먼저 데뷔했나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 아이유는 언제 데뷔했나요?
  중간 답변: 아이유는 2008년에 데뷔했습니다.
  추가 질문: 수지는 언제 데뷔했나요?
  중간 답변: 수지는 미쓰에이로 2010년에 데뷔했습니다.
  최종 답변은: 아이유
  

Question:
삼성전자 창업자의 아들이 졸업한 대학의 소재지는 어디인가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 삼성전자의 창업자는 누구인가요?
  중간 답변: 삼성전자의 창업자는 이병철입니다.
  추가 질문: 이병철의 아들은 누구인가요?
  중간 답변: 이병철의 아들 중 대표적인 인물은 이건희입니다.
  추가 질문: 이건희는 어느 대학을 졸업했나요?
  중간 답변: 이건희는 와세다대학을 졸업했습니다.
  추가 질문: 와세다대학은 어디에 있나요?
  중간 답변: 와세다대학은 일본 도쿄에 있습니다.
  최종 답변은: 일본 도쿄
  

Question:
2002년 월드컵 개최국들의 현재 수도는 같은가요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 2002년 월드컵은 어느 나라에서 개최되었나요?
  중간 답변: 2002년 월드컵은 한국과 일본이 공동 개최했습니다.
  추가 질문: 한국의 수도는 어디인가요?
  중간 답변: 한국의 수도는 서울

In [16]:
# 결과 출력
answer = llm.stream(final_prompt)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: 하정수 수석과 하정우는 각각 몇 년생인가요?  
중간 답변: 하정수 수석은 1964년생이고, 하정우는 1986년생입니다.  
최종 답변은: 하정수 수석이 더 나이가 많습니다.

## Example Selector

In [8]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

examples = [
    {
        "question": "한국의 대표 음식은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 한국의 전통 음식에는 어떤 것들이 있나요?
  중간 답변: 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.
  추가 질문: 이 중 가장 대표적인 음식은 무엇인가요?
  중간 답변: 김치가 가장 대표적인 한국 음식입니다.
  최종 답변은: 김치
  """,
    },
    {
        "question": "일본의 전통 음식은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 일본의 대표적인 요리에는 어떤 것들이 있나요?
  중간 답변: 스시, 라멘, 우동, 덴푸라 등이 있습니다.
  추가 질문: 이 중 가장 유명한 전통 음식은 무엇인가요?
  중간 답변: 스시가 가장 유명한 일본 전통 음식입니다.
  최종 답변은: 스시
  """,
    },
    {
        "question": "파이썬에서 가장 많이 사용되는 웹 프레임워크는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 파이썬의 웹 프레임워크에는 어떤 것들이 있나요?
  중간 답변: Django, Flask, FastAPI, Pyramid 등이 있습니다.
  추가 질문: 이 중 가장 널리 사용되는 것은 무엇인가요?
  중간 답변: Django가 가장 많이 사용되는 파이썬 웹 프레임워크입니다.
  최종 답변은: Django
  """,
    },
    {
        "question": "자바스크립트의 인기 있는 프론트엔드 프레임워크는 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 자바스크립트 프론트엔드 프레임워크에는 어떤 것들이 있나요?
  중간 답변: React, Vue, Angular, Svelte 등이 있습니다.
  추가 질문: 현재 가장 인기 있는 것은 무엇인가요?
  중간 답변: React가 현재 가장 인기 있는 프론트엔드 프레임워크입니다.
  최종 답변은: React
  """,
    },
    {
        "question": "서울에서 가장 높은 산은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 서울에 있는 주요 산들은 무엇인가요?
  중간 답변: 북한산, 남산, 관악산, 도봉산 등이 있습니다.
  추가 질문: 이 중 가장 높은 산은 무엇인가요?
  중간 답변: 북한산이 836.5m로 가장 높습니다.
  최종 답변은: 북한산
  """,
    },
    {
        "question": "부산의 유명한 해수욕장은 어디인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 부산에는 어떤 해수욕장들이 있나요?
  중간 답변: 해운대, 광안리, 송정, 다대포 해수욕장 등이 있습니다.
  추가 질문: 이 중 가장 유명한 곳은 어디인가요?
  중간 답변: 해운대 해수욕장이 가장 유명합니다.
  최종 답변은: 해운대 해수욕장
  """,
    },
    {
        "question": "머신러닝에서 가장 기본적인 알고리즘은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 머신러닝의 기초 알고리즘에는 어떤 것들이 있나요?
  중간 답변: 선형 회귀, 로지스틱 회귀, 결정 트리, SVM 등이 있습니다.
  추가 질문: 이 중 가장 기본적이고 먼저 배우는 것은 무엇인가요?
  중간 답변: 선형 회귀가 가장 기본적인 알고리즘입니다.
  최종 답변은: 선형 회귀
  """,
    },
    {
        "question": "딥러닝에서 이미지 분류에 주로 사용되는 모델은 무엇인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 이미지 분류에 사용되는 딥러닝 모델에는 어떤 것들이 있나요?
  중간 답변: CNN, ResNet, VGG, EfficientNet 등이 있습니다.
  추가 질문: 이 중 가장 널리 사용되는 기본 모델은 무엇인가요?
  중간 답변: CNN(Convolutional Neural Network)이 가장 기본적이고 널리 사용됩니다.
  최종 답변은: CNN
  """,
    },
    {
        "question": "한국에서 가장 오래된 대학은 어디인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 한국의 초기 대학들은 언제 설립되었나요?
  중간 답변: 연세대(1885년), 이화여대(1886년), 성균관대(1398년/1946년) 등이 있습니다.
  추가 질문: 현대적 의미의 대학 중 가장 오래된 곳은 어디인가요?
  중간 답변: 연세대학교(연희전문)가 1885년에 설립되어 가장 오래되었습니다.
  최종 답변은: 연세대학교
  """,
    },
    {
        "question": "미국의 명문 대학 중 가장 오래된 곳은 어디인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 미국의 초기 대학들은 언제 설립되었나요?
  중간 답변: 하버드(1636년), 예일(1701년), 프린스턴(1746년) 등이 있습니다.
  추가 질문: 이 중 가장 먼저 설립된 대학은 어디인가요?
  중간 답변: 하버드 대학교가 1636년에 설립되어 가장 오래되었습니다.
  최종 답변은: 하버드 대학교
  """,
    },
]

이 예시들은 다음과 같은 특징을 가지고 있습니다:

1. 의미적 유사성이 높은 쌍들:
- 한국/일본 음식 (1,2번)
- 파이썬/자바스크립트 프레임워크 (3,4번)
- 서울/부산 명소 (5,6번)
- 머신러닝/딥러닝 (7,8번)
- 한국/미국 대학 (9,10번)

2. SemanticSimilarityExampleSelector는 질문과 가장 유사한 예시들만 선택하므로, "한국 음식"을 물으면 "일본 음식"
예시도 함께 선택될 가능성이 높습니다.

3. MaxMarginalRelevanceExampleSelector는 유사하면서도 다양한 예시를 선택하므로, "한국 음식"을 물으면 음식 관련
  예시 하나와 다른 주제의 예시들을 함께 선택할 가능성이 높습니다.

In [9]:
test_questions = [
    {
        "category": "음식 (한국/일본)",
        "question": "중국의 대표적인 요리는 무엇인가요?",
        "expected_behavior": """
          - SemanticSimilarityExampleSelector: 한국 음식과 일본 음식 예시를 모두 선택할 가능성이 높음
          - MaxMarginalRelevanceExampleSelector: 음식 관련 예시 1개와 다른 주제의 예시들을 선택
          """,
    },
    {
        "category": "프로그래밍 프레임워크 (파이썬/자바스크립트)",
        "question": "자바에서 널리 사용되는 백엔드 프레임워크는 무엇인가요?",
        "expected_behavior": """
          - SemanticSimilarityExampleSelector: 파이썬과 자바스크립트 프레임워크 예시를 모두 선택
          - MaxMarginalRelevanceExampleSelector: 프레임워크 예시 1개와 다양한 다른 예시들을 선택
          """,
    },
    {
        "category": "도시 명소 (서울/부산)",
        "question": "제주도에서 가장 유명한 관광지는 어디인가요?",
        "expected_behavior": """
          - SemanticSimilarityExampleSelector: 서울 산과 부산 해수욕장 예시를 모두 선택
          - MaxMarginalRelevanceExampleSelector: 지역 명소 예시 1개와 다른 분야의 예시들을 선택
          """,
    },
    {
        "category": "AI/ML (머신러닝/딥러닝)",
        "question": "자연어 처리에서 가장 많이 사용되는 모델은 무엇인가요?",
        "expected_behavior": """
          - SemanticSimilarityExampleSelector: 머신러닝과 딥러닝 알고리즘 예시를 모두 선택
          - MaxMarginalRelevanceExampleSelector: AI 관련 예시 1개와 다양한 주제의 예시들을 선택
          """,
    },
    {
        "category": "대학 (한국/미국)",
        "question": "영국에서 가장 오래된 대학은 어디인가요?",
        "expected_behavior": """
          - SemanticSimilarityExampleSelector: 한국과 미국의 오래된 대학 예시를 모두 선택
          - MaxMarginalRelevanceExampleSelector: 대학 관련 예시 1개와 다른 분야의 예시들을 선택
          """,
    },
]

In [ ]:
test_questions[0]["question"]  # "중국의 대표적인 요리는 무엇인가요?"
test_questions[1][
    "question"
]  # "자바에서 널리 사용되는 백엔드 프레임워크는 무엇인가요?"
test_questions[2]["question"]  # "제주도에서 가장 유명한 관광지는 어디 인가요?"
test_questions[3]["question"]  # "자연어 처리에서 가장 많이 사용되는 모델은 무엇인가요?"
test_questions[4]["question"]  # "영국에서 가장 오래된 대학은 어디인가요?"
# test_questions --- IGNORE ---

'영국에서 가장 오래된 대학은 어디인가요?'

In [ ]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,  # 의미적 유사성 예시 선택기
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS

# Vector DB 생성 (고유한 저장소 이름 사용)
import time

collection_name = f"example_selector_{int(time.time())}"
chroma = Chroma(collection_name, OpenAIEmbeddings())

# 의미적 유사성을 측정하기 위해 OpenAIEmbeddings을 사용하여 단어를 숫자로 변환
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    OpenAIEmbeddings(),  # 텍스트를 벡터로 변환하는 역할
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    Chroma,
    # FAISS,
    # 이것은 생성할 예시의 수입니다.
    k=2,
)

# question = test_questions[0]['question']
question = "중국의 대표 전통음식은 무엇인가요?"
# 입력과 가장 유사한 예시를 선택합니다.
selected_examples = example_selector.select_examples({"question": question})

print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


입력에 가장 유사한 예시:
중국의 대표 전통음식은 무엇인가요?

question:
한국의 대표 음식은 무엇인가요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 한국의 전통 음식에는 어떤 것들이 있나요?
  중간 답변: 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.
  추가 질문: 이 중 가장 대표적인 음식은 무엇인가요?
  중간 답변: 김치가 가장 대표적인 한국 음식입니다.
  최종 답변은: 김치
  
question:
일본의 전통 음식은 무엇인가요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 일본의 대표적인 요리에는 어떤 것들이 있나요?
  중간 답변: 스시, 라멘, 우동, 덴푸라 등이 있습니다.
  추가 질문: 이 중 가장 유명한 전통 음식은 무엇인가요?
  중간 답변: 스시가 가장 유명한 일본 전통 음식입니다.
  최종 답변은: 스시
  


### Chroma와 FAISS의 차이로 인해 결과가 다른 이유

#### 1. 인덱싱 방식의 차이

  ##### Chroma의 경우
  - 문서를 추가할 때 내부적으로 중복 제거나 병합이 발생할 수 있음
  - 메타데이터 처리 방식이 다름
  - 특히 from_examples() 메서드 사용 시 내부 구현이 다름

  ##### FAISS의 경우
  - 각 문서를 독립적인 벡터로 저장
  - 순수한 벡터 유사도 검색에 최적화
  - 중복 처리 없이 모든 예시를 개별적으로 인덱싱

#### 2. from_examples() 메서드 구현의 차이

  **문제가 된 Chroma 코드에서 일어난 일** <br>
  ``` python
  example_selector = SemanticSimilarityExampleSelector.from_examples( 
      examples, 
      OpenAIEmbeddings(), 
      Chroma,  # 클래스를 전달 
      k=2, 
  )
  ```

  **내부적으로 Chroma는:**
  1. 컬렉션 생성 시 동일한 텍스트를 중복으로 처리할 수 있음
  2. 메타데이터 저장 방식이 달라 같은 예시가 중복 저장될 수 있음
  3. 벡터 저장소의 특성
<br>

  **Chroma 특성**
  - 영구 저장소 지향 (SQLite/DuckDB 백엔드)
  - 문서 ID 관리가 복잡
  - 동일 문서 중복 방지 메커니즘이 있지만 때로 오작동
<br>

  **FAISS 특성**
  - 메모리 기반 임시 저장소
  - 단순한 벡터 인덱스
  - 추가한 순서대로 정확히 저장

**결론** <br>
FAISS는 단순하고 예측 가능한 벡터 검색을 제공하는 반면, Chroma는 더 복잡한 문서 관리 시스템을 가지고 있어 from_examples() 같은 헬퍼 메서드 사용 시 예상치 못한 동작이 발생할 수 있습니다.

In [ ]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,  # 의미적 유사성 예시 선택기
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS

# Vector DB 생성 (고유한 저장소 이름 사용)
import time

collection_name = f"example_selector_{int(time.time())}"
chroma = Chroma(collection_name, OpenAIEmbeddings())

# 의미적 유사성을 측정하기 위해 OpenAIEmbeddings을 사용하여 단어를 숫자로 변환
# example_selector = SemanticSimilarityExampleSelector.from_examples(
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    OpenAIEmbeddings(),  # 텍스트를 벡터로 변환하는 역할
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    # Chroma,
    FAISS,
    # 이것은 생성할 예시의 수입니다.
    k=2,
)

# question = test_questions[0]['question']
question = "중국의 대표 전통음식은 무엇인가요?"
# 입력과 가장 유사한 예시를 선택합니다.
selected_examples = example_selector.select_examples({"question": question})

print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


입력에 가장 유사한 예시:
중국의 대표 전통음식은 무엇인가요?

question:
한국의 대표 음식은 무엇인가요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 한국의 전통 음식에는 어떤 것들이 있나요?
  중간 답변: 김치, 불고기, 비빔밥, 떡볶이 등이 있습니다.
  추가 질문: 이 중 가장 대표적인 음식은 무엇인가요?
  중간 답변: 김치가 가장 대표적인 한국 음식입니다.
  최종 답변은: 김치
  
question:
한국에서 가장 오래된 대학은 어디인가요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
  추가 질문: 한국의 초기 대학들은 언제 설립되었나요?
  중간 답변: 연세대(1885년), 이화여대(1886년), 성균관대(1398년/1946년) 등이 있습니다.
  추가 질문: 현대적 의미의 대학 중 가장 오래된 곳은 어디인가요?
  중간 답변: 연세대학교(연희전문)가 1885년에 설립되어 가장 오래되었습니다.
  최종 답변은: 연세대학교
  
